In [ ]:
import pandas as pd
df = pd.read_csv('top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.to_dict('records')

In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [ ]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [ ]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [ ]:
# Create collection to store wines
qdrant.create_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

In [ ]:
# vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [ ]:
user_prompt = "Suggest an amazing Malbec wine from Argentina"

In [ ]:
# Search time for awesome wines!
hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)

for hit in hits:
  print(hit.payload, "score:", hit.score)

In [ ]:
# Search time for awesome wines!
hits = qdrant.query_points(
    collection_name="top_wines",
    query=encoder.encode(user_prompt).tolist(),
    limit=3
)

for hit in hits.points:
    print(round(hit.score, 3), hit.payload)

In [ ]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits.points]

In [ ]:
# Now time to connect to the local large language model
# ./llava-v1.5-7b-q4.llamafile


from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message.content)